<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">TSS - Temporal Similarity Search : POC complete DATASET WITH API - avec enregistrement fichier .json volumineux et fichier parquet petit (sur le local pas de API) mais trop lourd dans la recherche victorielle</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
pip install kdbai-client

In [1]:
import pandas as pd
import numpy as np
# Charger les données
df = pd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

<h3 style="text-align: center; font-family: Arial, sans-serif; color: RED;">################# DÉCLARATION / INITIALISATION #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [2]:
# Convertir timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
#display(df.dtypes)

continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]

# Conserver uniquement les colonnes continues, catégorielles et 'timestamp'
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features
df = df[columns_to_keep]

display(df.head(2))

,timestamp,panne,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,0,-0.012,-0.024,53.600,0.04,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,0,-0.014,-0.022,53.675,0.04,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0


<h3 style="text-align: center; font-family: Arial, sans-serif; color: RED;">#################### FIN DÉCLARATION #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [3]:
import kdbai_client as kdbai
session = kdbai.Session(endpoint="https://cloud.kdb.ai/instance/atr0v2owym", api_key="a5b897a7b4-Av/lT//S3W++g8K11vVKfJo5rIj5gK492sbpts/1f+cr/5m5n5JluPcqcQR2gGiiQQ7SrCFAc7ma2RhT")

# create reference to default database
db = session.database('default')

In [6]:
print(db.tables)
# Return: an empty array if no tables have been created []

[]


In [14]:
##################################
###### SUPPRIMER LES TABLES ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")
test_table = db.table("test_sensors")

# Supprimer mes 2 tables
try:
    train_table.drop()
    test_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
    print("Table 'test_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

Table 'train_sensors' supprimée avec succès.
Table 'test_sensors' supprimée avec succès.


In [ ]:
###################################################################
################ Train : Panne2 & Panne3 & Panne4   ################
################ Test  : Panne1           #########################
###################################################################


train_periods = [{'start': '2020-04-18 23:59:10', 'end': '2020-09-01 03:59:50'}]
test_periods  = [{'start': '2020-02-01 00:00:00', 'end': '2020-04-18 23:59:00'}]

train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Filtrer les périodes pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entraînement : {len(train_df)} lignes")
print(f"Test : {len(test_df)} lignes")


In [29]:
###################################################################################################
################ Train : Panne1   (1J ET DEMI)                                              #######
################ Test  : Panne4  (15mn + 15mn + 15mn)     #########################################
###################################################################################################


train_periods = [{'start': '2020-04-17 21:30:00', 'end': '2020-04-19 00:15:00'}]
test_periods  = [{'start': '2020-07-15 14:00:00', 'end': '2020-07-15 14:45:00'}]

train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Filtrer les périodes pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entrainement : {len(train_df)} lignes")
print("Données d'entraînement (plage de temps) :")
print(train_df['timestamp'].min(), train_df['timestamp'].max())
print("---------------------------")
print(f"Test : {len(test_df)} lignes")
print("Données de test (plage de temps) :")
print(test_df['timestamp'].min(), test_df['timestamp'].max())


Entrainement : 9631 lignes
Données d'entraînement (plage de temps) :
2020-04-17 21:30:00 2020-04-19 00:15:00
---------------------------
Test : 271 lignes
Données de test (plage de temps) :
2020-07-15 14:00:00 2020-07-15 14:45:00


In [30]:
from tqdm import tqdm

# Paramètres des fenêtres temporelles
window_size = 100  # Taille de la fenêtre
step_size = 1      # Pas de glissement
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Initialiser les compteurs pour les fenêtres invalides
invalid_train_count = 0
invalid_test_count = 0

# Générer les fenêtres pour l'entraînement avec suivi de la progression
train_windows = []
print("Génération des fenêtres pour l'entraînement...")
for i in tqdm(range(0, len(train_df) - window_size + 1, step_size)):
    # Extraire les données de la fenêtre
    window_data = train_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    if len(window_data) == window_size * len(numeric_columns):  # Validation de la longueur
        train_windows.append({
            "index": i,
            "timestamp": train_df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist()
        })
    else:
        invalid_train_count += 1  # Compter les fenêtres invalides

# Générer les fenêtres pour le test avec suivi de la progression
test_windows = []
print("\nGénération des fenêtres pour le test...")
for i in tqdm(range(0, len(test_df) - window_size + 1, step_size)):
    # Extraire les données de la fenêtre
    window_data = test_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    if len(window_data) == window_size * len(numeric_columns):  # Validation de la longueur
        test_windows.append({
            "index": i,
            "timestamp": test_df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist()
        })
    else:
        invalid_test_count += 1  # Compter les fenêtres invalides

# Afficher le nombre de fenêtres générées et invalides
print(f"\nFenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres invalides d'entraînement : {invalid_train_count}")
print(f"Fenêtres de test : {len(test_windows)}")
print(f"Fenêtres invalides de test : {invalid_test_count}")


Génération des fenêtres pour l'entraînement...


100%|████████████████████████████████████████████████████████████████████████████| 9532/9532 [00:08<00:00, 1099.33it/s]



Génération des fenêtres pour le test...


100%|██████████████████████████████████████████████████████████████████████████████| 172/172 [00:00<00:00, 1128.51it/s]


Fenêtres d'entraînement : 9532
Fenêtres invalides d'entraînement : 0
Fenêtres de test : 172
Fenêtres invalides de test : 0


In [ ]:
# Sauvegarder les fenetres localement sur le C

import json

# Convertir les fenêtres pour rendre les Timestamps sérialisables
def convert_windows(windows):
    for window in windows:
        window['timestamp'] = window['timestamp'].isoformat()  # Convertir en chaîne
    return windows

# Convertir et sauvegarder les fenêtres d'entraînement
train_windows_serializable = convert_windows(train_windows)

# Sauvegarder les fenêtres d'entraînement
with open("train_windows.json", "w") as train_file:
    json.dump(train_windows, train_file)
print("TSS_Fenêtres d'entraînement sauvegardées dans train_windows.json.")

# Convertir et sauvegarder les fenêtres de test
test_windows_serializable = convert_windows(test_windows)
# Sauvegarder les fenêtres de test
with open("test_windows.json", "w") as test_file:
    json.dump(test_windows, test_file)
print("TSS_Fenetres de test sauvegardées dans test_windows.json.")


In [32]:
##################################
###### SUPPRIMER LES TABLES ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")
test_table = db.table("test_sensors")

# Supprimer mes 2 tables
try:
    train_table.drop()
    test_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
    print("Table 'test_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

Table 'train_sensors' supprimée avec succès.
Table 'test_sensors' supprimée avec succès.


In [33]:
# Schéma pour les tables
sensor_schema = [
    {"name": "index", "type": "int64"},               # Identifiant unique
    {"name": "timestamp", "type": "datetime64[ns]"}, # Timestamp
    {"name": "sensor_data", "type": "float64s"}      # Données vectorielles
                ]

# Index pour la recherche vectorielle
indexes = [
    {
        "name": "flat_index",
        "type": "flat",
        "column": "sensor_data",
        "params": {"metric": "L2","dims": 500}
    }
         ]

# Créer les tables
train_table = db.create_table("train_sensors", schema=sensor_schema, indexes=indexes)
test_table = db.create_table("test_sensors", schema=sensor_schema, indexes=indexes)
print("Table 'train_sensors' créée avec succès.")
print("Table 'test_sensors' créée avec succès.")

Table 'train_sensors' créée avec succès.
Table 'test_sensors' créée avec succès.


In [34]:
from tqdm import tqdm
from kdbai_client import KDBAIException

# Fonction d'insertion des données
def insert_data(table, data_windows, batch_size=100):
    """
    Insère les données dans une table KDB.AI par lots avec une barre de progression.
    """
    total_batches = -(-len(data_windows) // batch_size)  # Calculer le nombre total de batches
    print(f"Insertion dans la table {table.name}... Total de batches : {total_batches}")
    
    with tqdm(total=total_batches, desc=f"Insertion dans {table.name}") as pbar:
        for batch_index, i in enumerate(range(0, len(data_windows), batch_size), start=1):
            batch = data_windows[i:i + batch_size]
            try:
                table.insert(batch)
                pbar.update(1)
            except KDBAIException as e:
                pbar.set_postfix({"Erreur": "Oui"})
                print(f"Erreur lors de l'insertion du batch {batch_index}/{total_batches}: {e}")
                for record in batch:
                    try:
                        table.insert([record])  # Tentative d'insertion individuelle pour débogage
                    except KDBAIException as single_error:
                        print(f"Erreur sur l'enregistrement {record['index']}: {single_error}")

# Insertion des données d'entraînement
insert_data(train_table, train_windows)

# Insertion des données de test
insert_data(test_table, test_windows)

print("Insertion des données terminée.")


Insertion dans la table train_sensors... Total de batches : 96


Insertion dans train_sensors: 100%|████████████████████████████████████████████████████| 96/96 [00:46<00:00,  2.06it/s]


Insertion dans la table test_sensors... Total de batches : 2


Insertion dans test_sensors: 100%|███████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.37it/s]

Insertion des données terminée.


In [36]:
# Compte le nombre de lignes en utilisant la colonne 'index'
print(f"Entrainement : {len(train_df)} lignes")

aggs = {
    "row_count": ["count", "index"],
    "min_timestamp": ["min", "timestamp"],
    "max_timestamp": ["max", "timestamp"],
}

query_result = train_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")
print("----------------------------")
print(f"Test : {len(test_df)} lignes")
query_result = test_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")


Entrainement : 9631 lignes
Nombre de lignes : 0    9532
Name: row_count, dtype: int64
Timestamp minimum : 0   2020-04-17 21:30:00
Name: min_timestamp, dtype: datetime64[ns]
Timestamp maximum : 0   2020-04-18 23:58:30
Name: max_timestamp, dtype: datetime64[ns]
----------------------------
Test : 271 lignes
Nombre de lignes : 0    172
Name: row_count, dtype: int64
Timestamp minimum : 0   2020-07-15 14:00:00
Name: min_timestamp, dtype: datetime64[ns]
Timestamp maximum : 0   2020-07-15 14:28:30
Name: max_timestamp, dtype: datetime64[ns]


In [44]:
# Exemple de prédiction avec une fenêtre de test
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="numpy")


# Sélectionner un vecteur de requête (par exemple, la première fenêtre de test)
query_vector = test_windows[0]["sensor_data"]

# Effectuer une recherche vectorielle dans la table d'entraînement
results = train_table.search(
    vectors={"flat_index": [query_vector]},
    n=5,  # Trouver les 5 motifs les plus similaires
)

# Afficher les résultats de la recherche
print("\nRésultats de recherche vectorielle :")
for i, row in results[0].iterrows():
    print(f"Index d'entraînement : {row['index']}, Distance : {row['__nn_distance']}")
    print(f"Timestamp correspondant : {row['timestamp']}")
    print(f"Motif similaire (extrait des données capteurs) : {row['sensor_data'][:5]} ...")

# Extraire les indices similaires pour les visualiser
similar_indices = results[0]['index']

# Visualiser le motif de test et les motifs similaires
plt.figure(figsize=(10, 6))

# Tracer le vecteur de test
plt.plot(query_vector, marker="o", linestyle="-", label="Motif de Test", color="blue")

# Parcourir les indices similaires
for idx in similar_indices:
    # Requête pour récupérer les vecteurs similaires
    result_row = results[0][results[0]['index'] == idx]
    similar_vector = result_row['sensor_data'].iloc[0]
    
    # Tracer les vecteurs similaires
    plt.plot(similar_vector, marker="o", linestyle="--", alpha=0.7, label=f"Similaire {idx}")

# Ajouter des étiquettes et une légende
plt.xlabel("Timestamp")
plt.ylabel("Valeurs des Capteurs")
plt.title("Comparaison des Motifs : Test vs Similaires")
plt.legend()
plt.grid(True)
plt.show()




Résultats de recherche vectorielle :
Index d'entraînement : 6241, Distance : 3817.813720703125
Timestamp correspondant : 2020-04-18 14:50:10
Motif similaire (extrait des données capteurs) : [ 9.02    2.022  77.025   5.6875  8.882 ] ...
Index d'entraînement : 6240, Distance : 3823.2158203125
Timestamp correspondant : 2020-04-18 14:50:00
Motif similaire (extrait des données capteurs) : [ 9.02    2.022  77.075   5.7425  8.876 ] ...
Index d'entraînement : 6239, Distance : 3830.301025390625
Timestamp correspondant : 2020-04-18 14:49:50
Motif similaire (extrait des données capteurs) : [ 8.662   1.916  77.025   5.7325  8.884 ] ...
Index d'entraînement : 6242, Distance : 3843.76953125
Timestamp correspondant : 2020-04-18 14:50:20
Motif similaire (extrait des données capteurs) : [ 9.02   2.024 76.9    5.685  8.886] ...
Index d'entraînement : 6238, Distance : 3847.65771484375
Timestamp correspondant : 2020-04-18 14:49:40
Motif similaire (extrait des données capteurs) : [ 7.532   1.588  76.875  

C:\Users\usermine\AppData\Local\Temp\ipykernel_7596\409578788.py:48: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
total_batches = len(train_windows) // batch_size + (len(train_windows) % batch_size > 0)
print(f"Nombre de batches attendu : {total_batches}")


In [ ]:
duplicate_rows = train_df[train_df.duplicated()]
print(f"Lignes dupliquées dans train_df : {len(duplicate_rows)}")


In [ ]:
query_result = train_table.query(select=["timestamp"])
db_min_timestamp = query_result["timestamp"].min()
db_max_timestamp = query_result["timestamp"].max()
print(f"Plage temporelle dans la base : {db_min_timestamp} à {db_max_timestamp}")
print(f"Plage temporelle locale : {train_df['timestamp'].min()} à {train_df['timestamp'].max()}")


<h3 style="text-align: center; font-family: Arial, sans-serif; color: BLUE;">################# QUELQUES VALIDATIONS #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
print(f"Premiere fenetre train:\n{train_windows[0]}")

In [ ]:
aggs = {
    "row_count": ["count", "index"],
    "min_timestamp": ["min", "timestamp"],
    "max_timestamp": ["max", "timestamp"],
}

query_result = train_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")

In [ ]:
# Comparer les timestamps entre train_df et KDB.ai
db_timestamps = pd.date_range(start="2020-04-17 21:30:00", end="2020-04-18 23:58:30", freq="10s")
missing_timestamps = train_df[~train_df["timestamp"].isin(db_timestamps)]

print(f"Lignes manquantes dans la base : {len(missing_timestamps)}")
print(missing_timestamps)


In [ ]:
# Préparer les lignes manquantes pour l'insertion
missing_rows = []
for row in missing_timestamps.itertuples(index=True, name="Row"):
    missing_rows.append({
        "index": row.Index,  # Utiliser l'index fourni par itertuples
        "timestamp": row.timestamp,
        "sensor_data": list(row[2:])  # Inclure les colonnes numériques sous forme de liste
    })

# Insérer les données manquantes dans la table train_sensors
batch_size = 10  # Taille du batch pour l'insertion
print(f"Réinsertion des {len(missing_rows)} lignes manquantes...")

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(missing_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données manquantes terminée.")


In [ ]:
# Vérifiez les colonnes utilisées pour la création de sensor_data
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]  # Exemple
print("Colonnes utilisées pour sensor_data :", numeric_columns)

# Préparer les lignes manquantes pour l'insertion
missing_rows = []
for row in missing_timestamps.itertuples(index=True, name="Row"):
    # Vérifiez les données avant de les ajouter
    try:
        sensor_data = [getattr(row, col) for col in numeric_columns]  # Extraire les colonnes nécessaires
        missing_rows.append({
            "index": row.Index,  # Utiliser l'index fourni par itertuples
            "timestamp": row.timestamp,
            "sensor_data": sensor_data  # Liste de données numériques
        })
    except AttributeError as e:
        print(f"Erreur lors de la préparation des données pour l'index {row.Index} : {e}")

# Réinsertion dans la base
batch_size = 10
print(f"Réinsertion des {len(missing_rows)} lignes manquantes...")

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(missing_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données manquantes terminée.")


In [ ]:
# Vérification des dimensions de chaque vecteur avant l'insertion
expected_length = 500  # Longueur attendue des vecteurs

invalid_vectors = []
for row in missing_rows:
    if len(row["sensor_data"]) != expected_length:
        invalid_vectors.append(row)

# Si des vecteurs invalides sont trouvés, les afficher
if invalid_vectors:
    print(f"Vecteurs invalides détectés : {len(invalid_vectors)}")
    for idx, invalid_row in enumerate(invalid_vectors[:5]):  # Afficher les 5 premiers vecteurs invalides
        print(f"Index : {invalid_row['index']}, Longueur : {len(invalid_row['sensor_data'])}")
else:
    print("Tous les vecteurs ont la bonne dimension.")

# Réinsertion uniquement des vecteurs valides
valid_rows = [row for row in missing_rows if len(row["sensor_data"]) == expected_length]

print(f"Réinsertion des {len(valid_rows)} lignes valides...")
for i in range(0, len(valid_rows), batch_size):
    batch = valid_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(valid_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données valides terminée.")


In [ ]:
# Taille réelle de la première fenêtre
print(f"Taille des données dans la première fenêtre d'entraînement : {len(train_windows[0]['sensor_data'])}")
print(f"Taille des données dans la première fenêtre de test : {len(test_windows[0]['sensor_data'])}")

# Cela correspond à window_size (100) × nombre de colonnes (numeric_columns, soit 5).

print('--------------------------------')
# Vérifier la taille des fenêtres générées
for i in range(5):  # Tester sur les 5 premières fenêtres
    actual_window = train_df.iloc[i:i + window_size][numeric_columns]
    print(f"Fenêtre {i} - Taille réelle : {len(actual_window)}")
    

In [ ]:
# Initialiser des compteurs pour détecter les erreurs
invalid_index_count = 0
invalid_timestamp_count = 0
invalid_sensor_data_count = 0

# Parcourir toutes les fenêtres pour vérifier les types et formats
for i, window in enumerate(train_windows):
    # Vérifier l'index
    if not isinstance(window['index'], int):
        print(f"Fenêtre {i} : 'index' non valide ({type(window['index'])})")
        invalid_index_count += 1
    
    # Vérifier le timestamp
    try:
        pd.Timestamp(window['timestamp'])  # Vérifie si le timestamp est convertible
    except Exception as e:
        print(f"Fenêtre {i} : 'timestamp' non valide ({window['timestamp']})")
        invalid_timestamp_count += 1
    
    # Vérifier sensor_data
    if not isinstance(window['sensor_data'], list) or len(window['sensor_data']) != 500:
        print(f"Fenêtre {i} : 'sensor_data' non valide (Longueur : {len(window['sensor_data'])})")
        invalid_sensor_data_count += 1
    elif not all(isinstance(value, float) for value in window['sensor_data']):
        print(f"Fenêtre {i} : 'sensor_data' contient des valeurs non flottantes.")
        invalid_sensor_data_count += 1

# Résumé des erreurs détectées
print(f"Erreurs dans 'index' : {invalid_index_count}")
print(f"Erreurs dans 'timestamp' : {invalid_timestamp_count}")
print(f"Erreurs dans 'sensor_data' : {invalid_sensor_data_count}")

In [ ]:
# Tester l'insertion avec un petit lot

# Schéma pour les tables
sensor_schema = [
    {"name": "index", "type": "int64"},               # Identifiant unique
    {"name": "timestamp", "type": "datetime64[ns]"}, # Timestamp
    {"name": "sensor_data", "type": "float64s"}      # Données vectorielles
]

# Index pour la recherche vectorielle
indexes = [
    {
        "name": "flat_index",
        "type": "flat",
        "column": "sensor_data",
        "params": {"metric": "L2","dims": 500}
    }
]

# Créer les tables
train_table = db.create_table("train_sensors", schema=sensor_schema, indexes=indexes)
test_table = db.create_table("test_sensors", schema=sensor_schema, indexes=indexes)


single_window = [train_windows[0]]  # Utiliser uniquement la première fenêtre
try:
    train_table.insert(single_window)
    print("Insertion réussie pour une seule fenêtre.")
except Exception as e:
    print(f"Erreur lors de l'insertion d'une seule fenêtre : {e}")

In [ ]:
from datetime import datetime

try:
    datetime.fromisoformat(single_window[0]['timestamp'])
    print("Le format du timestamp est correct.")
except ValueError as e:
    print(f"Problème avec le format du timestamp : {e}")


In [ ]:
for value in single_window[0]['sensor_data']:
    if not isinstance(value, float):
        print(f"Problème détecté : {value} de type {type(value)}")


In [ ]:
if len(single_window[0]['sensor_data']) != 500:
    print("La longueur de sensor_data n'est pas correcte.")


In [ ]:
print("Schéma de la table :")
print(train_table.schema)


In [ ]:
from datetime import datetime

for window in train_windows:
    window['timestamp'] = datetime.fromisoformat(window['timestamp'])


In [ ]:
train_table.query(limit=5)  # Affiche les 5 premières lignes


In [ ]:
test_row_count = test_table.query(aggs=aggs)
print(f"Lignes stockées dans la table de test : {test_row_count}")


In [ ]:
# Afficher le contenu du lot envoyé pour inspection
print("Contenu du lot de test :")
for i, window in enumerate(test_batch):
    print(f"Fenêtre {i}: {window}")


In [ ]:
# Vérification approfondie des valeurs
for i, window in enumerate(test_batch):
    if not all(isinstance(v, float) and not (np.isnan(v) or np.isinf(v)) for v in window['sensor_data']):
        print(f"Fenêtre {i} contient des valeurs invalides.")


In [ ]:
# Remplacer les valeurs invalides par une valeur par défaut (ex. : 0.0)
for window in train_windows:
    window['sensor_data'] = [
        0.0 if (np.isnan(v) or np.isinf(v)) else v
        for v in window['sensor_data']
    ]


In [ ]:
# Vérifier la structure des 5 premières fenêtres
for window in train_windows[:5]:
    print(f"Index : {window['index']}")
    print(f"Timestamp : {window['timestamp']}")
    print(f"Longueur de sensor_data : {len(window['sensor_data'])}")
    print(f"Exemple de données : {window['sensor_data'][:5]}")


In [ ]:
# Vérification des longueurs des fenêtres
for i, window in enumerate(train_windows):
    if len(window['sensor_data']) != window_size * len(numeric_columns):
        print(f"Fenêtre {i} a une taille incorrecte : {len(window['sensor_data'])}")
print("Validation de la longueur des fenêtres terminée.")

In [ ]:
# Vérification de la continuité temporelle
for i, window in enumerate(train_windows):
    timestamps = train_df.iloc[window['index']:window['index'] + window_size]['timestamp']
    time_diffs = timestamps.diff().dt.total_seconds().dropna()
    if not all(time_diffs == 10):  # Vérifie si tous les gaps sont de 10 secondes
        print(f"Fenêtre {i} a des timestamps non consécutifs.")
print("Validation de la continuité temporelle terminée.")

In [ ]:
# Exemple : Vérifier la réponse brute
response = train_table.insert(train_windows[:5])
print("Réponse brute :", response)

In [ ]:
print(db.tables)

In [ ]:
# Requête pour compter les lignes dans une table
train_row_count = db.query("SELECT COUNT(*) AS row_count FROM train_sensors").iloc[0]['row_count']
print(f"Lignes stockées dans la table d'entraînement : {train_row_count}")

test_row_count = db.query("SELECT COUNT(*) AS row_count FROM test_sensors").iloc[0]['row_count']
print(f"Lignes stockées dans la table de test : {test_row_count}")


In [ ]:
# Afficher toutes les méthodes et attributs de l'objet db
print(dir(db.tables))

In [ ]:
# Afficher toutes les méthodes et attributs de l'objet train_table
print(dir(train_table))

In [ ]:
# Obtenir les détails sur la méthode query
help(train_table.query)

In [ ]:
# Définir l'agrégation pour compter les lignes
aggs = {'row_count': ['count', 'index']}  # Compte le nombre de lignes en utilisant la colonne 'index'

# Effectuer la requête pour compter les lignes
train_row_count = train_table.query(aggs=aggs)
print(f"Lignes stockées dans la table d'entraînement : {train_row_count}")


In [ ]:
# Parametres des fenetres temporelles
window_size = 100  # Taille de la fenetre
step_size = 1      # Pas de glissement
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Générer les fenetres pour l'entrainement
train_windows = [
    {
        "index": i,
        "timestamp": train_df.iloc[i]["timestamp"],
        "sensor_data": train_df.iloc[i:i + window_size][numeric_columns].values.flatten().tolist()
    }
    for i in range(0, len(train_df) - window_size + 1, step_size)
]

# Générer les fenetres pour le test
test_windows = [
    {
        "index": i,
        "timestamp": test_df.iloc[i]["timestamp"],
        "sensor_data": test_df.iloc[i:i + window_size][numeric_columns].values.flatten().tolist()
    }
    for i in range(0, len(test_df) - window_size + 1, step_size)
]

print(f"Fenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres de test : {len(test_windows)}")


In [ ]:
from sklearn.decomposition import PCA

numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Construire les vecteurs pour chaque fenetre (numeric_columns)
window_vectors = np.array([
    window[numeric_columns].values.flatten() for window in windows
])

# Appliquer PCA pour réduire la dimensionnalité
pca = PCA(n_components=8)  # Réduire a 8 dimensions
compressed_vectors = pca.fit_transform(window_vectors)

print(f"Dimensions des vecteurs compressés : {compressed_vectors.shape}")
print(f"Vecteur compressé pour la première fenêtre : {compressed_vectors[0]}")